# Temporal Plots of DESI Telemetry Data

As is, the DESI telemetry data is spread out across multiple different tables. As is is shown in the correlations plots in the `correlation` folder, there us very weak correlation across different tables. Before proper analysis can be done, the several datatables of interest must 
be syncronized in time. This involves rebinning and interpolation of the data is currently being hosted by NERSC.

## Imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import trange, tqdm
from IPython.display import display, Image
import pytz
try:
    import numba as nb
    nb.cuda.detect()
    import cupy as cp
    %pip install -q cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
    import cudf as cd
    gpu_access = True
except:
    gpu_access = False
    print('No GPU devices available!')
data_dir = '../../py/desiforecast/data'

No GPU devices available!


## Load Data

In [99]:
with open(f'{data_dir}/environmentmonitor_tower.pkl', 'rb') as pf:
    tower_data = pd.read_pickle(pf)
with open(f'{data_dir}/environmentmonitor_dome.pkl', 'rb') as pf:
    dome_data = pd.read_pickle(pf)
with open(f'{data_dir}/environmentmonitor_telescope.pkl', 'rb') as pf:
    telescope_data = pd.read_pickle(pf)
with open(f'{data_dir}/etc_seeing.pkl', 'rb') as pf:
    seeing_data = pd.read_pickle(pf)
with open(f'{data_dir}/etc_telemetry.pkl', 'rb') as pf:
    telemetry_data = pd.read_pickle(pf)
with open(f'{data_dir}/tcs_info.pkl', 'rb') as pf:
    tcs_data = pd.read_pickle(pf)
with open(f'{data_dir}/temp_delta.pkl', 'rb') as pf:
    temp_delta_data = pd.read_pickle(pf)
telemetry_data.sort_index(inplace=True)
seeing_data.sort_index(inplace=True)

In [100]:
print(tower_data.notnull().all())
print(dome_data.notnull().all())
print(telescope_data.notnull().all())
print(seeing_data.notnull().all())
print(telemetry_data.notnull().all())
print(tcs_data.notnull().all())
print(temp_delta_data.notnull().all())

temperature       True
pressure          True
humidity          True
wind_speed        True
wind_direction    True
dimm              True
dewpoint          True
dtype: bool
dome_left_upper     True
dome_left_lower     True
dome_right_upper    True
dome_right_lower    True
dome_back_upper     True
dome_back_lower     True
dome_floor_ne       True
dome_floor_nw       True
dome_floor_s        True
dtype: bool
mirror_avg_temp        True
mirror_desired_temp    True
mirror_temp            True
mirror_cooling         True
air_temp               True
air_flow               True
air_dewpoint           True
dtype: bool
etc_seeing     True
seeing        False
dtype: bool
seeing          False
transparency    False
skylevel        False
dtype: bool
mirror_ready    True
airmass         True
dtype: bool
mins                   True
maxs                   True
mean                   True
median                 True
std                   False
var                   False
start                  True
en

In [84]:
print(temp_delta_data.columns.values)

['mins' 'maxs' 'mean' 'median' 'std' 'var' 'start' 'end' 'equalize_time'
 'ambient_when_equal']


In [108]:
from astropy.stats import scott_bin_width
from math import trunc

no_outliers = remove_outliers(temp_delta_data['mean'])
print(no_outliers.shape)
print(len(no_outliers))
months = np.linspace(1, 12, 12, dtype=np.int64)

def yearly_delta_stats():
    years = [2019, 2020, 2021, 2022, 2023]
    for i in years:
        for j in months:
            valid = np.argwhere(np.logical_and(temp_delta_data.index[no_outliers].year == i, temp_delta_data.index[no_outliers].month == j)).flatten()
            if len(valid):
                directory = f'time_plots/temp_statistics/{i}'
                if not os.path.isdir(directory):
                    os.makedirs(directory)
                fig, ax = plt.subplots(figsize=(20,15))
                ax.set_title('Daily Temperature Statistics vs. Date/Time')
                ax.set_xlabel('Day')
                ax.autoscale(True)
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['mins'].iloc[no_outliers][valid], label='daily minimum')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['maxs'].iloc[no_outliers][valid], label='daily maximum')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['mean'].iloc[no_outliers][valid], label='daily mean')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['median'].iloc[no_outliers][valid], label='daily median')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['std'].iloc[no_outliers][valid], label='daily std')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['var'].iloc[no_outliers][valid], label='daily var')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['start'].iloc[no_outliers][valid], label='daily start temp')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['end'].iloc[no_outliers][valid], label='daily end temp')
                ax.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['ambient_when_equal'].iloc[no_outliers][valid], label='daily ambient when temp equalized')
                ax_handles, _ = ax.get_legend_handles_labels()
                twin = ax.twinx()
                twin.set_ylabel('Time When Temps Equalize')
                twin.autoscale(True)
                twin.plot(temp_delta_data.index[no_outliers][valid], temp_delta_data['equalize_time'].iloc[no_outliers][valid], c='maroon', label='time to equalize')
                twin_handles, _ = twin.get_legend_handles_labels()
                ax.legend(handles=ax_handles+twin_handles)
                plt.rcParams['figure.facecolor'] = 'white'
                fig.savefig(f'{directory}/{i}-{j}.png')
                plt.close(fig)

yearly_delta_stats()

(698,)
698


### print(temp_delta_data['equalize_time'])

## 02/09/23 Tasks
    - Plot for every month
        - Get delta temp when mirror cooling is off
            - 1d distribution for each month
            - 3d plot of all months
        - Get time it reach desired temp during mirror cooling
            - as a function of ambient temperature at start
            - as a function of air temperature variance
            - percent of time outside of ideal window
        - Ambient air during mirror cooling    
    - how often operators follow forecasts of the night
    - how accurate are the forecasts and to what degree
    - March 8, desi movie in Oakland hills

## Single Day Temperature Plots (Night vs Day)

In [6]:
@nb.jit(nopython=False)
def new_temperature_plot(year=2022, month=9, day=24, offset=1, save=False):
    start_date = pd.Timestamp(year=year,month=month,day=day,hour=17, minute=0, second=0, microsecond=0, tz="America/Phoenix")-pd.DateOffset(day=1)
    end_date = start_date+pd.DateOffset(day=offset)
    if gpu_access:
        start_date = start_date.to_numpy()
        end_date = end_date.to_numpy()

    print(f'Start date: {start_date}')
    print(f'Offset length (# of days): {offset}')
    
    fig, ax = plt.subplots(ncols=2, figsize=(20,10))
    ax0_plots = []
    ax1_plots = []
    ax[0].autoscale(True)
    ax[0].set_title('Mirror Temperature & Seeing vs. Date/Time')
    ax[0].set_ylabel('Mirror Temperature (Celcius)')
    ax[0].set_xlabel('Date/Time')
    ax[1].autoscale(True)
    ax[1].set_title('Dome & Tower Temperature vs. Date/Time')
    ax[1].set_ylabel('Dome Temperature (Celcius)')
    ax[1].set_xlabel('Date/Time')

    if gpu_access:
        telescope_indicies = cp.argwhere(cp.logical_and(telescope_data.index >= start_date, telescope_data.index <= end_date)).flatten()
    else:
        telescope_indicies = np.argwhere(np.logical_and(telescope_data.index >= start_date, telescope_data.index <= end_date)).flatten()
    temp_delta = remove_outliers(telescope_data.iloc[telescope_indicies]['air_temp']-telescope_data.iloc[telescope_indicies]['mirror_temp'])
    print(f'Number of valid telescope environment samples in current time window: {len(telescope_indicies)}')
    if len(telescope_indicies):
        ax0_plots.append(ax[0].plot(telescope_data.index[telescope_indicies], telescope_data.iloc[telescope_indicies]['mirror_avg_temp'], label='average mirror temp')[0])
        ax0_plots.append(ax[0].plot(telescope_data.index[telescope_indicies], telescope_data.iloc[telescope_indicies]['mirror_temp'], label='mirror temp')[0])
        ax0_plots.append(ax[0].plot(telescope_data.index[telescope_indicies], telescope_data.iloc[telescope_indicies]['mirror_desired_temp'], label='desired mirror temp')[0])
        ax0_plots.append(ax[0].plot(telescope_data.index[telescope_indicies], telescope_data.iloc[telescope_indicies]['air_temp'], label='air temp')[0])
        ax0_plots.append(ax[0].plot(telescope_data.index[telescope_indicies], telescope_data.iloc[telescope_indicies]['mirror_cooling'], label='mirror cooling')[0])
        ax0_plots.append(ax[0].plot(temp_delta.index, temp_delta, label='temp delta')[0])
        ax[0].axhline(y=1.5, color='maroon', linestyle='--')
        ax[0].axhline(y=-1.5, color='maroon', linestyle='--')
        ax[0].annotate(u"1.5 \u2103", xy=(end_date, 1.75), annotation_clip=False, ha='right')
        ax[0].annotate(u"-1.5 \u2103", xy=(end_date, -1.25), annotation_clip=False, ha='right')

    if gpu_access:
        telemetry_indicies = cp.argwhere(cp.logical_and(telemetry_data.index >= start_date, telemetry_data.index <= end_date)).flatten()
    else:
        telemetry_indicies = np.argwhere(np.logical_and(telemetry_data.index >= start_date, telemetry_data.index <= end_date)).flatten()
    print(f'Number of valid etc telemetry samples in current time window: {len(telemetry_indicies)}')
    if len(telemetry_indicies):
        twin0 = ax[0].twinx()
        ax0_plots.append(twin0.plot(telemetry_data.index[telemetry_indicies], telemetry_data.iloc[telemetry_indicies]['seeing'], color="indigo", label='seeing')[0])
        twin0.autoscale(True)
        twin0.set_ylabel('Seeing')

    if gpu_access:
        dome_indicies = cp.argwhere(cp.logical_and(dome_data.index >= start_date, dome_data.index <= end_date)).flatten()
    else:
        dome_indicies = np.argwhere(np.logical_and(dome_data.index >= start_date, dome_data.index <= end_date)).flatten()
    print(f'Number of valid dome environment samples in current time window: {len(dome_indicies)}')
    if len(dome_indicies):
        ax1_plots.append(ax[1].plot(dome_data.index[dome_indicies], dome_data.iloc[dome_indicies]['dome_floor_ne'], label='dome floor temp (NE)')[0])
        ax1_plots.append(ax[1].plot(dome_data.index[dome_indicies], dome_data.iloc[dome_indicies]['dome_floor_nw'], label='dome floor temp (NW)')[0])
        ax1_plots.append(ax[1].plot(dome_data.index[dome_indicies], dome_data.iloc[dome_indicies]['dome_floor_s'], label='dome floor temp (S)')[0])

    if gpu_access:
        tower_indicies = cp.argwhere(cp.logical_and(tower_data.index >= start_date, tower_data.index <= end_date)).flatten()
    else:
        tower_indicies = np.argwhere(np.logical_and(tower_data.index >= start_date, tower_data.index <= end_date)).flatten()
    print(f'Number of valid tower environment samples in current time window: {len(tower_indicies)}')
    if len(tower_indicies):
        twin1 = ax[1].twinx()
        ax1_plots.append(twin1.plot(tower_data.index[tower_indicies], tower_data.iloc[tower_indicies]['temperature'], color="indigo", label='tower temp')[0])
        twin1.autoscale(True)
        twin1.set_ylabel('Tower Temperature (Celcius)')
    
    if ax1_plots:
        ax[1].legend(handles=ax1_plots)
    else:
        ax[1].remove()

    if ax0_plots:
        ax[0].legend(handles=ax0_plots)
        if save:
            plt.savefig(f'time_plots/with_temp_delta/{year}-{month}-{day}_{offset}-day-offset.png', format='png')
        plt.show()
    else:
        print("Insufficient data in this time window!")

In [41]:
# removes results outside of 5 std
@nb.jit(forceobj=True, parallel=True)
def remove_outliers(data):
    return np.argwhere(np.abs(stats.zscore(data.to_numpy())) < 5).flatten()

In [97]:
@nb.jit(forceobj=True)
def get_time_ranges(cooling_on):
    changes = telescope_data['mirror_cooling'][1:].to_numpy()-telescope_data['mirror_cooling'][:len(telescope_data)-1].to_numpy()
    ranges = []
    start=0
    for i in range(len(changes)):
        if cooling_on:
            if changes[i] == 1:
                start = i
            elif changes[i] == -1:
                end = i
                ranges.append(np.linspace(start+1, end, end-start, dtype=np.int64))
        else:
            if changes[i] == -1:
                start = i
            elif changes[i] == 1:
                end = i
                ranges.append(np.linspace(start+1, end, end-start, dtype=np.int64))
    return ranges

def mirror_cooling_plots(cooling_on):
    time_ranges = get_time_ranges(cooling_on=cooling_on)
    mirror_ambient_delta = telescope_data['air_temp']-telescope_data['mirror_temp']
    mirror_desired_delta = telescope_data['mirror_desired_temp']-telescope_data['mirror_temp']
    folder = 'mirror_cooling_on' if cooling_on else 'mirror_cooling_off'
    df = pd.DataFrame()
    mins = []
    maxs = []
    mean = []
    median = []
    std = []
    var = []
    start = []
    end = []
    start_time = []
    equalize_time = []
    ambient_when_equal = []
    for i in trange(len(time_ranges), desc=folder):
        if cooling_on:
            mins.append(mirror_ambient_delta.iloc[time_ranges[i]].min())
            maxs.append(mirror_ambient_delta.iloc[time_ranges[i]].max())
            mean.append(mirror_ambient_delta.iloc[time_ranges[i]].mean())
            median.append(mirror_ambient_delta.iloc[time_ranges[i]].median())
            std.append(mirror_ambient_delta.iloc[time_ranges[i]].std())
            var.append(mirror_ambient_delta.iloc[time_ranges[i]].var())
            start.append(mirror_ambient_delta.iloc[time_ranges[i][0]])
            end.append(mirror_ambient_delta.iloc[time_ranges[i][-1]])
            start_time.append(mirror_ambient_delta.index[time_ranges[i]][0])
            equalize = np.argwhere(mirror_ambient_delta.iloc[time_ranges[i]].to_numpy()<1e-2).flatten()
            if len(equalize):
                time = (mirror_ambient_delta.index[time_ranges[i]][equalize[-1]]-mirror_ambient_delta.index[time_ranges[i]][-1]).seconds/3600
                print(time)
                equalize_time.append(time)
                ambient_when_equal.append(telescope_data.iloc[time_ranges[i]]['air_temp'][equalize[-1]])
            else:
                equalize_time.append(None)
                ambient_when_equal.append(None)
                
#         fig, ax = plt.subplots(ncols=2, figsize=(20,10))
#         ax0_plots = []
#         ax1_plots = []
#         ax[0].autoscale(True)
#         ax[0].set_title('Mirror Temperature vs. Date/Time')
#         ax[0].set_ylabel('Mirror Temperature (Celcius)')
#         ax[0].set_xlabel('Date/Time')
#         ax[1].autoscale(True)
#         ax[1].set_title('Dome & Tower Temperature vs. Date/Time')
#         ax[1].set_ylabel('Dome Temperature (Celcius)')
#         ax[1].set_xlabel('Date/Time')
    
#         fig.autofmt_xdate()        

#         ax0_plots.append(ax[0].plot(telescope_data.index[time_ranges[i]], telescope_data.iloc[time_ranges[i]]['mirror_avg_temp'], label='average mirror temp')[0])
#         ax0_plots.append(ax[0].plot(telescope_data.index[time_ranges[i]], telescope_data.iloc[time_ranges[i]]['mirror_temp'], label='mirror temp')[0])
#         ax0_plots.append(ax[0].plot(telescope_data.index[time_ranges[i]], telescope_data.iloc[time_ranges[i]]['mirror_desired_temp'], label='desired mirror temp')[0])
#         ax0_plots.append(ax[0].plot(telescope_data.index[time_ranges[i]], telescope_data.iloc[time_ranges[i]]['air_temp'], label='air temp')[0])
#         ax0_plots.append(ax[0].plot(telescope_data.index[time_ranges[i]], telescope_data.iloc[time_ranges[i]]['mirror_cooling'], label='mirror cooling')[0])
#         ax0_plots.append(ax[0].plot(mirror_ambient_delta.index[time_ranges[i]], mirror_ambient_delta.iloc[time_ranges[i]], label='ambient & mirror temp delta')[0])
#         ax0_plots.append(ax[0].plot(mirror_desired_delta.index[time_ranges[i]], mirror_desired_delta.iloc[time_ranges[i]], label='desired & mirror temp delta')[0])
#         ax[0].axhline(y=1.5, color='maroon', linestyle='--')
#         ax[0].axhline(y=-1.5, color='maroon', linestyle='--')
#         ax[0].annotate(u"1.5 \u2103", xy=(telescope_data.index[time_ranges[i]][-1], 1.75), annotation_clip=False, ha='right')
#         ax[0].annotate(u"-1.5 \u2103", xy=(telescope_data.index[time_ranges[i]][-1], -1.25), annotation_clip=False, ha='right')

#         if not cooling_on:
#             telemetry_indicies = np.argwhere(np.logical_and(telemetry_data.index >= telescope_data.index[time_ranges[i]][0], telemetry_data.index <= telescope_data.index[time_ranges[i]][-1])).flatten()
#             if len(telemetry_data.iloc[telemetry_indicies]['seeing']):
#                 twin0 = ax[0].twinx()
#                 ax[0].set_title('Mirror Temperature & Seeing vs. Date/Time')
#                 twin0.set_ylabel('Seeing')
#                 twin0.autoscale(True)
#                 ax0_plots.append(twin0.plot(telemetry_data.index[telemetry_indicies], telemetry_data.iloc[telemetry_indicies]['seeing'], color="indigo", label='seeing')[0])

#         ax1_plots.append(ax[1].plot(dome_data.index[time_ranges[i]], dome_data.iloc[time_ranges[i]]['dome_floor_ne'], label='dome floor temp (NE)')[0])
#         ax1_plots.append(ax[1].plot(dome_data.index[time_ranges[i]], dome_data.iloc[time_ranges[i]]['dome_floor_nw'], label='dome floor temp (NW)')[0])
#         ax1_plots.append(ax[1].plot(dome_data.index[time_ranges[i]], dome_data.iloc[time_ranges[i]]['dome_floor_s'], label='dome floor temp (S)')[0])

#         twin1 = ax[1].twinx()
#         ax1_plots.append(twin1.plot(tower_data.index[time_ranges[i]], tower_data.iloc[time_ranges[i]]['temperature'], color="indigo", label='tower temp')[0])
#         twin1.autoscale(True)
#         twin1.set_ylabel('Tower Temperature (Celcius)')

#         ax[0].legend(handles=ax0_plots)
#         ax[1].legend(handles=ax1_plots)

#         year = telescope_data.index[time_ranges[i]][0].year
#         month = telescope_data.index[time_ranges[i]][0].month
#         day = telescope_data.index[time_ranges[i]][0].day
#         plt.rcParams['figure.facecolor'] = 'white'
#         fig.savefig(f'time_plots/{folder}/{year}-{month}-{day}.png', format='png')
#         plt.close(fig)
        
    df['mins'] = mins
    df['maxs'] = maxs
    df['mean'] = mean
    df['median'] = median
    df['std'] = std
    df['var'] = var
    df['start'] = start
    df['end'] = end
    df['start_time'] = start_time
    df['equalize_time'] = equalize_time
    df['ambient_when_equal'] = ambient_when_equal
    df.set_index('start_time', inplace=True)
    with open(f'{data_dir}/temp_delta.pkl', 'wb') as pf:
        pickle.dump(df, pf)

In [ ]:
mirror_cooling_plots(cooling_on=True)
# mirror_cooling_plots(cooling_on=False)

In [ ]:
print(np.max(telescope_data['mirror_temp']))

In [30]:
ambient_mirror_delta = remove_outliers(telescope_data['air_temp']-telescope_data['mirror_temp'])
ambient_temp = remove_outliers(telescope_data['air_temp'])
print(len(ambient_mirror_delta))
print(len(ambient_temp))


20469907
20469908


## Monthly Temperature Delta Distributions

In [ ]:
from copy import deepcopy
ambient_mirror_delta = remove_outliers(telescope_data['air_temp']-telescope_data['mirror_temp'])
ambient_temp = remove_outliers(telescope_data['air_temp'])
years = [2019, 2020, 2021, 2022, 2023]
months = np.linspace(1, 12, 12, dtype=np.int64)
ambient_distributions = {}
for i in years:
    year = {}
    for j in months:
        year[j] = []
    ambient_distributions[i] = year
delta_distributions = deepcopy(ambient_distributions)
for i in trange(len(ambient_mirror_delta), desc='binning temperature delta by year and month'):
    ambient_distributions[ambient_temp.index[i].year][ambient_temp.index[i].month].append(ambient_temp.iloc[i])
    delta_distributions[ambient_mirror_delta.index[i].year][ambient_mirror_delta.index[i].month].append(ambient_mirror_delta.iloc[i])
with open(f'{data_dir}/ambient_distributions.pkl', 'wb') as pf:
    pickle.dump(ambient_distributions, pf)
with open(f'{data_dir}/delta_distributions.pkl', 'wb') as pf:
    pickle.dump(delta_distributions, pf)

In [36]:
from scipy.stats import skewnorm
from astropy.stats import scott_bin_width
from math import trunc
with open(f'{data_dir}/ambient_distributions.pkl', 'rb') as pf:
    ambient_distributions = pickle.load(pf)
with open(f'{data_dir}/delta_distributions.pkl', 'rb') as pf:
    delta_distributions = pickle.load(pf)
def monthly_delta_distribution():
    for i in years:
        directory = f'time_plots/temp_distributions/{i}'
        if not os.path.isdir(directory):
            os.makedirs(directory)
        for j in tqdm(months, desc=f'{i}'):
            if len(delta_distributions[i][j]):
                fig, ax = plt.subplots(ncols=2, figsize=(15,5))
                
                bins = trunc(np.ptp(delta_distributions[i][j])/scott_bin_width(delta_distributions[i][j]))
                ax[0].hist(delta_distributions[i][j], bins=bins, density=True, label='histogram')
                xs = np.linspace(np.min(delta_distributions[i][j]), np.max(delta_distributions[i][j]), 10000)
                ys = skewnorm.pdf(xs, *skewnorm.fit(delta_distributions[i][j]))
                ax[0].scatter(xs, ys, 5, color='r', marker='.', label='skewed guassian fit')
                ax[0].set_title(f'Distribution of Ambient-Mirror Temperatuere Delta ({i}-{j})')
                ax[0].set_ylabel('Density (%)')
                ax[0].set_xlabel('Temperature Delta')
                ax[0].legend()
                ax[0].autoscale(True)                

                bins = trunc(np.ptp(ambient_distributions[i][j])/scott_bin_width(ambient_distributions[i][j]))
                ax[1].hist(ambient_distributions[i][j], bins=bins, density=True, label='histogram')
                xs = np.linspace(np.min(ambient_distributions[i][j]), np.max(ambient_distributions[i][j]), 10000)
                ys = skewnorm.pdf(xs, *skewnorm.fit(ambient_distributions[i][j]))
                ax[1].scatter(xs, ys, 5, color='r', marker='.', label='skewed guassian fit')
                ax[1].set_title(f'Distribution of Ambient Temperature ({i}-{j})')
                ax[1].set_ylabel('Density (%)')
                ax[1].set_xlabel('Ambient Temperature')
                ax[1].legend()
                ax[1].autoscale(True)                
                
                fig.savefig(f'{directory}/{i}-{j}.png', format='png') 
                plt.close(fig)

In [37]:
monthly_delta_distribution()

2019:   0%|          | 0/12 [00:00<?, ?it/s]

2020:   0%|          | 0/12 [00:00<?, ?it/s]

2021:   0%|          | 0/12 [00:00<?, ?it/s]

2022:   0%|          | 0/12 [00:00<?, ?it/s]

2023:   0%|          | 0/12 [00:00<?, ?it/s]

In [13]:
print(np.ptp(telescope_data['mirror_temp']))

29.463


In [ ]:
with open(f'{data_dir}/temp_delta.pkl', 'rb') as pf:
    temp_delta_data = pd.read_pickle(pf)

In [ ]:
print(temp_delta_data['equalize_time'])

In [ ]:
time_ranges1 = get_time_ranges(find_mirror_on=True)
time_ranges2 = get_time_ranges(find_mirror_on=False)
assert(time_ranges1!=time_ranges2)

In [ ]:
with open(f'{data_dir}/environmentmonitor_tower.pkl', 'wb') as pf:
    pickle.dump(tower_data, pf)
with open(f'{data_dir}/environmentmonitor_dome.pkl', 'wb') as pf:
    pickle.dump(dome_data, pf)
with open(f'{data_dir}/environmentmonitor_telescope.pkl', 'wb') as pf:
    pickle.dump(telescope_data, pf)
with open(f'{data_dir}/etc_seeing.pkl', 'wb') as pf:
    pickle.dump(seeing_data, pf)
with open(f'{data_dir}/etc_seeing.pkl', 'wb') as pf:
    pickle.dump(telemetry_data, pf)
with open(f'{data_dir}/etc_seeing.pkl', 'wb') as pf:
    pickle.dump(tcs_data, pf)

In [ ]:
temp_delta = (telescope_data['air_temp']-telescope_data['mirror_temp']).dropna()
print(f'mininum: {temp_delta.min()}')
print(f'maximum: {temp_delta.max()}')
print(f'variance: {temp_delta.var()}')
print(f'standard deviation: {temp_delta.std()}')

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(temp_delta.index, temp_delta, label='temperature delta')
ax.set_title('Temperature Delta (Celcius) vs. Date & Time')
ax.set_ylabel('Temperature Delta (Celcius)')
ax.set_xlabel('Date & Time')
fig.autofmt_xdate()
plt.legend()
plt.rcParams['figure.facecolor'] = 'white'
plt.savefig('temp_delta.png', format='png')
plt.show()

In [ ]:
temp_delta = remove_outliers(telescope_data['air_temp']-telescope_data['mirror_temp'])
print(f'mininum: {temp_delta.min()}')
print(f'maximum: {temp_delta.max()}')
print(f'variance: {temp_delta.var()}')
print(f'standard deviation: {temp_delta.std()}')

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(temp_delta.index, temp_delta, label='temperature delta')
ax.set_title('Temperature Delta (Celcius) vs. Date & Time')
ax.set_ylabel('Temperature Delta (Celcius)')
ax.set_xlabel('Date & Time')
fig.autofmt_xdate()
plt.legend()
plt.rcParams['figure.facecolor'] = 'white'
plt.savefig('temp_delta_no_outliers.png', format='png')
plt.show()

In [ ]:
time_diff = np.zeros((len(telemetry_indicies)))
for i in range(len(telemetry_indicies)-1):
    time_diff[i] = (telemetry_data.index[telemetry_indicies[i+1]]-telemetry_data.index[telemetry_indicies[i]]).total_seconds()
plt.scatter(telemetry_data.index[telemetry_indicies], time_diff)
plt.show()
rsm = np.sqrt(np.sum(np.square(time_diff))/len(time_diff))
print(time_diff)
print(len(time_diff))
print(rsm)
print(np.median(time_diff))
print(np.mean(time_diff))
print(np.min(time_diff))
print(np.max(time_diff))


## Data Visualization Before Syncrization

In [ ]:
new_temperature_plot()

In [ ]:
new_temperature_plot(year=2022, month=6, day=10, offset=1, save=True)

In [ ]:
new_temperature_plot(year=2022, month=6, day=10, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=6, day=10, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=1, day=1, offset=7, save=True)

In [ ]:
new_temperature_plot(year=2021, month=12, day=27, offset=7, save=True)

In [ ]:
new_temperature_plot(year=2022, month=1, day=21, offset=7, save=True)

In [ ]:
new_temperature_plot(year=2021, month=2, day=1, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=2, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=6, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=7, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=8, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=9, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=10, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=11, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=12, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=3, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=4, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2021, month=5, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=1, day=15, offset=7, save=True)

In [ ]:
new_temperature_plot(year=2022, month=2, day=1, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=1, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=1, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=2, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=2, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=3, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=3, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=4, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=4, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=5, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=5, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=6, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=6, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=7, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=7, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=8, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=8, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=9, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=9, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=10, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=10, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=11, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=11, day=15, offset=5, save=True)

In [ ]:
new_temperature_plot(year=2022, month=12, day=15, offset=3, save=True)

In [ ]:
new_temperature_plot(year=2022, month=12, day=15, offset=5, save=True)

In [ ]:
display(Image(filename='correlation/temp_corr_before_sync.png'))